In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time

In [2]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
endpoint = "https://lab3-albertregus.cognitiveservices.azure.com/"
vision_key = "570a3afbac67498ab7a45c74ec317fa9"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(vision_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

# Problema 1

## 1. a) calitatea procesului de recunoastere a textului, atat la nivel de caracter, cat si la nivel de cuvant prin folosirea unei metrici de distanta

In [3]:
# img = open("test1.png", "rb")
img = open("images/test2.jpeg", "rb")
# img = open("images/firefox.png", "rb")

read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            # print(line.text)
            result.append(line.text)

print(result)

ComputerVisionOcrErrorException: Operation returned an invalid status code 'PermissionDenied'

In [10]:
# groundTruth = ["Google Cloud", "Platform"]
groundTruth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoaree de", "Inteligenta Artificiala!"]
# groundTruth = ["Firefox"]

noOfCorrectLines = sum(i == j for i, j in zip(result, groundTruth))
print(noOfCorrectLines)

0


In [11]:
original_text = "Success in rezolvarea tEMELOR la LABORA toarele de Inteligenta Artificiala!";

# original_text = "Firefox"

- calitatea procesului de recunoastere a textului folosind distanta Levenshtein

In [12]:
import editdistance

def ocr_quality(original_text, ocr_output):
    distance = editdistance.eval(original_text, ocr_output)
    return distance

ocr_text = ' '.join(result)
print(ocr_text)

distance = ocr_quality(original_text, ocr_text)
print("Distanta Levenshtein: ", distance)

Lucces in resolvarea TEMELOR la LABORA toarele de Inteligenta Artificialà!
Distanta Levenshtein:  5


- calitatea procesului de recunoastere a textului folosing distanta Hamming

In [13]:
def hamming_distance(s1, s2):

    if len(s1) != len(s2):
        raise ValueError("Stringurile trebuie sa aiba aceeasi lungime")
    
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

ocr_text = ' '.join(result)

try:
    distance = hamming_distance(original_text, ocr_text)
    print("Distanta Hamming: ", distance)
except ValueError as error:
    print(error)

Stringurile trebuie sa aiba aceeasi lungime


- calitatea procesului de recunoastere a textului folosing distanta Jaro-Winkler <br>
0 - stringurile sunt total diferite <br>
1 - stringurile sunt la fel

In [14]:
import jellyfish

ocr_text = ' '.join(result)

jaro_winkler_distance = jellyfish.jaro_winkler_similarity(original_text, ocr_text)

print("Distanta Jaro-Winkler: ", jaro_winkler_distance)

Distanta Jaro-Winkler:  0.8654223237321829


## 2) calitatea localizarii corecte a textului in imagine

In [15]:
import pytesseract
from pytesseract import Output
import cv2

In [16]:
image = cv2.imread("images/firefox.png")

if image is not None:
    cv2.imshow("Image", cv2.resize(image, (800, 600)))
    cv2.waitKey(0)  
    cv2.destroyAllWindows() 
else:
    print("Image not found or cannot be read.")

rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = pytesseract.image_to_data(rgb, output_type=Output.DICT)

print(results)
print(results["text"], results["conf"])

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/albert/.local/lib/python3.10/site-packages/cv2/qt/plugins"


{'level': [1, 2, 3, 4, 5, 5], 'page_num': [1, 1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1, 1], 'par_num': [0, 0, 1, 1, 1, 1], 'line_num': [0, 0, 0, 1, 1, 1], 'word_num': [0, 0, 0, 0, 1, 2], 'left': [0, 32, 32, 32, 32, 128], 'top': [0, 41, 41, 41, 41, 65], 'width': [334, 229, 229, 229, 78, 133], 'height': [153, 81, 81, 81, 81, 32], 'conf': [-1, -1, -1, -1, 43, 96], 'text': ['', '', '', '', '*', 'Firefox']}
['', '', '', '', '*', 'Firefox'] [-1, -1, -1, -1, 43, 96]


In [17]:
for i in range(0, len(results["text"])):
	
	x = results["left"][i]
	y = results["top"][i]
	w = results["width"][i]
	h = results["height"][i]

	text = results["text"][i]   # extrage textul 
	conf = int(results["conf"][i])  # nivelul de recunoastere

	if conf >= 50:

		print("Confidence: {}".format(conf))
		print("Text: {}".format(text))
		print("")

		text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
		cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1)
		cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

Confidence: 96
Text: Firefox



In [18]:
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 3) posibilitati de imbunatatire a recunoasterii textului

In [19]:
image = cv2.imread("images/firefox.png")
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results1 = pytesseract.image_to_data(rgb, output_type=Output.DICT)


- utilizand Gaussian Blur

In [21]:
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
results2 = pytesseract.image_to_data(blurred_image, output_type=Output.DICT)

cv2.imshow('Original Image', image)
cv2.imshow('Blurred Image', blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
ocr_text = ' '.join(results2["text"])
text = ' '.join(results1["text"])

print(text)
print(ocr_text)

    Mucee! a Ae solvate  CTEMELOR fa.    APA GORA Ae 2 e : ay         
    hee d fe sobvarea  PEMELOR 4a    AA GORA tone af    


- utilizand Median Filter

In [20]:
median_blurred_image = cv2.medianBlur(image, 3)
results2 = pytesseract.image_to_data(median_blurred_image, output_type=Output.DICT)

cv2.imshow('Original Image', image)
cv2.imshow('Blurred Image', blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'blurred_image' is not defined

In [24]:
ocr_text = ' '.join(results2["text"])
text = ' '.join(results1["text"])

print(text)
print(ocr_text)

    Mucee! a Ae solvate  CTEMELOR fa.    APA GORA Ae 2 e : ay         
    Zz CceA im fie solvareeg  tEMELOR Ae    AA GORA oe s    


- folosind grayscale

In [25]:
grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
results2 = pytesseract.image_to_data(grayscale_image, output_type=Output.DICT)

cv2.imshow('Original Image', image)
cv2.imshow('Blurred Image', blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
ocr_text = ' '.join(results2["text"])
text = ' '.join(results1["text"])

print(text)
print(ocr_text)

    Mucee! a Ae solvate  CTEMELOR fa.    APA GORA Ae 2 e : ay         
    Doan Si fe sobvareg  PEMELOR 4a.    LAGORA ten pele = of    
